# Table of Contents --> TBU

1. [Global parameters](#1-bullet) <br>
    <br>
    
2. [Loading datas](#2-bullet) <br>
    <br>

3. [Preprocessing](#3-bullet) <br>
    I - [Cleaning](#4-bullet) <br>
    II - [Split train/test and preprocessing](#5-bullet) <br>
    III - [Dimensionality reduction](#6-bullet) <br>
    IV - [Creation of folds for cv](#7-bullet) <br>
    <br>

4. [Baseline model testing](#8-bullet) <br>
    I - [Dummy classifiers ](#9-bullet) <br>
    II - [Quick testing](#10-bullet) <br>
    <br>

5. [Cross-validation model testing](#11-bullet) <br>
    I - [Linear models](#12-bullet) <br>
    II - [KNN](#13-bullet) <br>
    III - [SVM](#14-bullet) <br>
    IV - [Trees and ensemblist methods](#15-bullet) <br>
    V - [Neural networks](#16-bullet) <br>
    VI - [Compare](#17-bullet) <br>
    <br>
    
6. [Selected model fine tuning](#18-bullet) <br>
    I - [xx](#19-bullet) <br>
    II - [xx](#20-bullet) <br>
    III - [xx](#21-bullet) <br>
    <br>

7. [Features importance](#xx-bullet) <br>
    I - [xx](#xx-bullet) <br>
    II - [xx](#xx-bullet) <br>
    III - [xx](#xx-bullet) <br>
    <br>

# 1. Global parameters <a class="anchor" id="1-bullet"></a>

In [198]:
# General input
random_state = 50 

# Cross-validation
optimized_metric = 'roc_auc' 
num_folds = 5
stratified = True

# 2. Loading datas <a class="anchor" id="2-bullet"></a>

In [199]:
# Classic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import timeit
from contextlib import contextmanager

import warnings
from pandas.core.common import SettingWithCopyWarning
# warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

# Project specific functions
from P7_functions import *

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn import manifold, decomposition
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import imblearn

# Sklearn models
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Neural networks
import tensorflow as tf

# Evaluation
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal
from sklearn.dummy import DummyClassifier

In [200]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [201]:
baseline_data = pd.read_csv('./Clean_datas/baseline_data.csv', sep=",")
data = pd.read_csv('./Clean_datas/clean_data_1.csv', sep=",")

In [202]:
data.head()

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CURRENT_LOAN_LTV,CURRENT_LOAN_INCOME_CREDIT_PERC,CURRENT_LOAN_PAYMENT_RATE,TOTAL_AMT_ANNUITY,TOTAL_AMT_CREDIT,TOTAL_EFFORT_RATE,TOTAL_INCOME_CREDIT_PERC,TOTAL_PAYMENT_RATE,DAYS_EMPLOYED_PERC,INCOME_PER_PERSON
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...,1.158397,0.498036,0.060749,247829.081500,888586.065,1.223847,0.227890,0.278903,0.067329,202500.0
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,...,1.145199,0.208736,0.027598,292122.185803,2103502.500,1.081934,0.128357,0.138874,0.070862,135000.0
2,2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,1.000000,0.500000,0.050000,NaN,NaN,NaN,NaN,NaN,0.011814,67500.0
3,3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,...,1.052803,0.431748,0.094941,NaN,NaN,NaN,NaN,NaN,0.159905,67500.0
4,4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,...,1.000000,0.236842,0.042623,NaN,NaN,NaN,NaN,NaN,0.152418,121500.0


In [203]:
data.drop(['Unnamed: 0', 'SK_ID_CURR'], axis=1, inplace=True)

In [204]:
y = data['TARGET']
x = data.drop(['TARGET'], axis=1)
baseline_y = baseline_data['TARGET']
baseline_x = baseline_data.drop(['TARGET'], axis=1) # Note : categorical data already encoded

In [205]:
print(x.shape)
print(y.shape)

(307507, 399)
(307507,)


In [206]:
# Look targets breakdown
y.value_counts().apply(lambda x: x / y.count())

0    0.91927
1    0.08073
Name: TARGET, dtype: float64

We have very imbalanced classes, we will use StratifiedKFold for now. And see for SMOTE after

# 3. Preprocessing <a class="anchor" id="3-bullet"></a>

## I - Cleaning <a class="anchor" id="4-bullet"></a>

In [207]:
x.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CURRENT_LOAN_LTV,CURRENT_LOAN_INCOME_CREDIT_PERC,CURRENT_LOAN_PAYMENT_RATE,TOTAL_AMT_ANNUITY,TOTAL_AMT_CREDIT,TOTAL_EFFORT_RATE,TOTAL_INCOME_CREDIT_PERC,TOTAL_PAYMENT_RATE,DAYS_EMPLOYED_PERC,INCOME_PER_PERSON
count,307507.000000,3.075070e+05,3.075070e+05,307495.000000,3.072290e+05,307507.000000,307507.000000,252133.000000,307507.000000,307507.000000,...,307229.000000,307507.000000,307495.000000,2.163040e+05,2.163140e+05,216304.000000,216314.000000,216304.000000,252133.000000,3.075050e+05
mean,0.417047,1.687977e+05,5.990286e+05,27108.666786,5.383977e+05,0.020868,-16037.027271,-2384.142254,-4986.131376,-2994.201670,...,1.122994,0.399669,0.053695,9.411559e+05,1.926691e+06,5.708165,0.154313,0.609689,0.156860,9.310608e+04
std,0.722119,2.371246e+05,4.024926e+05,14493.798379,3.694472e+05,0.013831,4363.982424,2338.327666,3522.883030,1509.454566,...,0.124036,0.507927,0.022481,5.921754e+06,2.459518e+06,33.373152,0.246091,3.542178,0.133548,1.013739e+05
min,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,...,0.150000,0.011801,0.022073,3.006000e+03,4.500000e+04,0.003830,0.000603,0.001404,-0.000000,2.812500e+03
25%,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-3175.000000,-7479.500000,-4299.000000,...,1.000000,0.193802,0.036900,9.718556e+04,7.524000e+05,0.640621,0.072865,0.093214,0.056098,4.725000e+04
50%,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1648.000000,-4504.000000,-3254.000000,...,1.118800,0.306272,0.050000,3.004183e+05,1.305000e+06,2.029257,0.116145,0.211615,0.118733,7.500000e+04
75%,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-767.000000,-2010.000000,-1720.000000,...,1.198000,0.495376,0.064043,7.142913e+05,2.254457e+06,4.373481,0.189675,0.395364,0.219167,1.125000e+05
max,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,0.000000,0.000000,0.000000,...,6.000000,208.003328,0.124430,6.802079e+08,3.356847e+08,3702.839475,95.097017,264.392053,0.728811,3.900000e+07


In [208]:
# Defining numerical and categorical columns
categorical_cols = [col for col in x.columns if x[col].dtype == 'object']
numerical_cols = list(x.drop(categorical_cols, axis=1).columns)

In [209]:
# Checking infinite values
  
count = np.isinf(x[numerical_cols]).values.sum()
print("The df contains " + str(count) + " infinite values")

The df contains 8877 infinite values


In [210]:
# We replace inf values by NaN
x.replace([np.inf, -np.inf], np.nan, inplace=True)

## II - Split train/test and preprocessing <a class="anchor" id="5-bullet"></a>

In [211]:
# Split between train and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=random_state)

In [212]:
# Definition of preprocessing steps

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('stdscaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [213]:
# Preprocess datas
x_train_processed = preprocessor.fit_transform(x_train)
x_test_processed = preprocessor.transform(x_test)

## III - Dimensionality reduction <a class="anchor" id="6-bullet"></a>

To speed up our algorithms on our model selection, we will reduce the dimensionality of our dataset 

In [214]:
# PCA on processed data

print("Dimensions x_train before PCA reduction : ", x_train_processed.shape)
print("Dimensions x_test before PCA reduction : ", x_test_processed.shape)
pca = decomposition.PCA(n_components=0.99)

print("")
with timer("Proceed PCA on train and test set"):
    x_train_pca = pca.fit_transform(x_train_processed)
    x_test_pca = pca.transform(x_test_processed)

print("Dimensions x_train after PCA reduction : ", x_train_pca.shape)
print("Dimensions x_test after PCA reduction : ", x_test_pca.shape)

Dimensions x_train before PCA reduction :  (246005, 670)
Dimensions x_test before PCA reduction :  (61502, 670)

Proceed PCA on train and test set - done in 21s
Dimensions x_train after PCA reduction :  (246005, 289)
Dimensions x_test after PCA reduction :  (61502, 289)


## IV - Creation of folds for cv <a class="anchor" id="7-bullet"></a>

In [215]:
folds = create_folds(x_train_pca, y_train, num_folds=num_folds, stratified=True, random_state=random_state)

In [245]:
# Look classs balance in each fold

for i in range(5):
   print(Counter(y_train.iloc[folds[i][1]]))

Counter({0: 45234, 1: 3967})
Counter({0: 45234, 1: 3967})
Counter({0: 45233, 1: 3968})
Counter({0: 45233, 1: 3968})
Counter({0: 45233, 1: 3968})


# 4. Baseline model testing <a class="anchor" id="8-bullet"></a>

## I - Dummy classifiers <a class="anchor" id="9-bullet"></a>

### a. Unbalanced

In [220]:
dummies = test_dummy_classifiers(x_train_pca, y_train, valid_size=0.2, 
                                 strategies_list=None, random_state=random_state, constant=0, balance_class=False)
dummies

,most_frequent,prior,stratified,uniform,constant
accuracy,0.919981,0.919981,0.853194,0.500376,0.919981
f1,0.000000,0.000000,0.079521,0.136565,0.000000
precision,0.000000,0.000000,0.079795,0.079240,0.000000
recall,0.000000,0.000000,0.079248,0.493777,0.000000
roc_auc,0.500000,0.500000,0.499879,0.500000,0.500000
cross_entropy,2.763748,2.763748,5.070557,17.256768,2.763748
fit_time,0.016626,0.014415,0.008039,0.012216,0.013446
predict_time,0.001482,0.000999,0.001896,0.001412,0.000870


In [221]:
print("Average roc_auc : {:.6f}".format(dummies.iloc[4].mean()))

Average roc_auc : 0.499976


### b. Balanced

In [255]:
dummies_2 = test_dummy_classifiers(x_train_pca, y_train, valid_size=0.2, 
                                   strategies_list=None, random_state=random_state, constant=0, balance_class=True)
dummies_2

,most_frequent,prior,stratified,uniform,constant
accuracy,0.919981,0.919981,0.642914,0.500376,0.919981
f1,0.000000,0.000000,0.128392,0.136565,0.000000
precision,0.000000,0.000000,0.079778,0.079240,0.000000
recall,0.000000,0.000000,0.328677,0.493777,0.000000
roc_auc,0.500000,0.500000,0.499461,0.500000,0.500000
cross_entropy,2.763748,2.763748,12.333564,17.256768,2.763748
fit_time,0.006022,0.001100,0.000928,0.001024,0.001601
predict_time,0.001698,0.000466,0.001782,0.000491,0.000528


In [256]:
print("Average roc_auc : {:.6f}".format(dummies_2.iloc[4].mean()))

Average roc_auc : 0.499892


## II - Quick testing <a class="anchor" id="10-bullet"></a>

### a. Unbalanced

In [271]:
# Test some models without hyperparameters optimization

models_list = [
    # 'GradientBoostingClassifier', 
    'RandomForestClassifier', 
    'KNeighborsClassifier',
    # 'GaussianProcessClassifier', 
    'LogisticRegression', 
    'RidgeClassifier', 
    # 'SGDClassifier',
    # 'LinearSVC', 
    # 'NuSVC', 
    'SVC', 
    # 'DecisionTreeClassifier'
]

with timer("Quick test of some classifiers"):
    quick_test_1 = quick_classifiers_test(x_train_pca, y_train, valid_size=0.2,models_list=models_list, 
                                          random_state=random_state, max_iter=10000, n_jobs=-1, balance_class=False)

quick_test_1

Quick test of some classifiers - done in 22795s


C:\Users\robin\anaconda3\envs\fastai\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,RandomForestClassifier,KNeighborsClassifier,LogisticRegression,RidgeClassifier,SVC
accuracy,0.919798,0.915103,0.920063,0.919981,0.919981
f1,0.000507,0.045693,0.055249,0.001015,0.000000
precision,0.090909,0.227273,0.508850,0.500000,0.000000
recall,0.000254,0.025400,0.029210,0.000508,0.000000
roc_auc,0.662433,0.577950,0.771007,0.767978,0.673001
cross_entropy,2.770066,2.932232,2.760942,2.763748,2.763748
fit_time,422.415992,0.126682,123.843777,0.732317,20675.629150
predict_time,1.262353,41.642139,0.040803,0.019151,743.403910
compute_score_time,0.070560,0.047518,0.046990,0.040441,0.045043


In [272]:
print("Average roc_auc : {:.6f}".format(quick_test_1.iloc[4].mean()))
print("Max roc_auc : {:.6f}".format(quick_test_1.iloc[4].max()))

Average roc_auc : 0.690474
Max roc_auc : 0.771007


### b. Balanced

In [262]:
with timer("Quick test of some classifiers"):
    quick_test_2 = quick_classifiers_test(x_train_pca, y_train, valid_size=0.2,models_list=models_list, 
                                          random_state=random_state, max_iter=10000, n_jobs=-1, balance_class=True)

quick_test_2

Quick test of some classifiers - done in 1464s


,RandomForestClassifier,KNeighborsClassifier,LogisticRegression,RidgeClassifier,SVC
accuracy,0.895043,0.687974,0.841345,0.848641,0.859962
f1,0.193629,0.194628,0.310668,0.311547,0.312101
precision,0.251317,0.122645,0.238121,0.244913,0.257115
recall,0.157480,0.471171,0.446787,0.427991,0.397003
roc_auc,0.714894,0.622611,0.770914,0.770269,0.763111
cross_entropy,3.625124,10.777218,5.479852,5.227829,4.836808
fit_time,60.890628,0.019599,18.908819,0.173946,584.349935
predict_time,0.926250,11.538366,0.018744,0.018717,386.572186
compute_score_time,0.036803,0.049717,0.037641,0.041494,0.055558


In [270]:
print("Average roc_auc : {:.6f}".format(quick_test_2.iloc[4].mean()))
print("Max roc_auc : {:.6f}".format(quick_test_2.iloc[4].max()))

Average roc_auc : 0.728360
Max roc_auc : 0.770914


Balancing seems to improve our performances, we will apply it on further testing

In [266]:
balance_class = True

# 5. Cross-validation model testing <a class="anchor" id="11-bullet"></a>

## I - Linear models <a class="anchor" id="12-bullet"></a>

In [267]:
# LogisticRegression

model = LogisticRegression(random_state=random_state, max_iter=10000)
param_grid = {'C' : np.linspace(0.1, 1, num=4)}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters on training set :
{'model__C': 1.0}
Best score on training set : 0.768
Proceed LogisticRegression - done in 45s


In [265]:
# RidgeClassifier

model = RidgeClassifier(random_state=random_state, max_iter=10000)
param_grid = {'alpha' : np.linspace(1, 10, num=4, dtype=int)}

with timer("Proceed RidgeClassifier"):
    RidgeClassifier_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                           param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters on training set :
{'model__alpha': 10}
Best score on training set : 0.768
Proceed RidgeClassifier - done in 31s


## II - KNN <a class="anchor" id="13-bullet"></a>

In [268]:
# KNeighborsClassifier

model = KNeighborsClassifier()
param_grid = {'n_neighbors' : np.linspace(3, 10, num=4, dtype=int)}

with timer("Proceed KNeighborsClassifier"):
    KNeighborsClassifier_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                                param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters on training set :
{'model__n_neighbors': 10}
Best score on training set : 0.646
Proceed KNeighborsClassifier - done in 215s


## III - SVM <a class="anchor" id="14-bullet"></a>

In [269]:
# LinearSVC

model = LinearSVC(random_state=random_state, max_iter=10000)
param_grid = {'C' : np.linspace(0.1, 1, num=4)}

with timer("Proceed LinearSVC"):
    LinearSVC_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                     param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters on training set :
{'model__C': 0.1}
Best score on training set : 0.768
Proceed LinearSVC - done in 2628s


C:\Users\robin\anaconda3\envs\fastai\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [273]:
# SVC

model = SVC(kernel='rbf', random_state=random_state, max_iter=10000)
param_grid = {'C' : np.linspace(0.1, 1, num=4)}

with timer("Proceed SVC"):
    SVC_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                               param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters on training set :
{'model__C': 0.4}
Best score on training set : 0.685
Proceed SVC - done in 3122s


C:\Users\robin\anaconda3\envs\fastai\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


## IV - Trees and ensemblist methods <a class="anchor" id="15-bullet"></a>

In [274]:
# DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=random_state)
param_grid = {'min_samples_split' : [2, 8], 'min_samples_leaf' : [1, 5]}

with timer("Proceed DecisionTreeClassifier"):
    DecisionTreeClassifier_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                                  param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters on training set :
{'model__min_samples_leaf': 5, 'model__min_samples_split': 2}
Best score on training set : 0.575
Proceed DecisionTreeClassifier - done in 177s


In [25]:
# GradientBoostingClassifier --> not tested in 2nd iteration (too long to train, 43891s iteration 1)

# model = GradientBoostingClassifier(random_state=random_state)
# param_grid = {'n_estimators' : [10, 100, 500]}

# with timer("Proceed GradientBoostingClassifier"):
    # GradientBoostingClassifier_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                                      # param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters on training set :
{'n_estimators': 500}
Best score on training set : 0.760
Proceed GradientBoostingClassifier - done in 43891s


In [275]:
# RandomForestClassifier

model = RandomForestClassifier(random_state=random_state)
param_grid = {'n_estimators' : [10, 500]}

with timer("Proceed RandomForestClassifier"):
    RandomForestClassifier_clf = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                                  param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters on training set :
{'model__n_estimators': 500}
Best score on training set : 0.729
Proceed RandomForestClassifier - done in 1642s


## V - Neural networks <a class="anchor" id="16-bullet"></a>

In [286]:
# We will test a standard binary classification NN model (activations hidden = relu, last activation = sigmoid) 
# and try to optimize hyperparameters (learning rate, nb neurons and layers)

def build_model(n_hidden=1, n_neurons=30, learning_rate=0.001, input_shape=[289]):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["AUC"])
    return model

In [287]:
keras_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [288]:
# Testing hyperparameters

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.linspace(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

# Testing model parameters
nn_clf = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, 
                            cv=folds, scoring=optimized_metric, random_state=random_state)

with timer("Proceed NeuralNetwork"):
    nn_clf.fit(x_train_pca, y_train, epochs=10, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10)])

Epoch 1/10
6151/6151 [==============================] - 10s 925us/step - loss: 0.2899 - auc: 0.5990
Epoch 2/10
6151/6151 [==============================] - 6s 897us/step - loss: 0.2634 - auc: 0.7017
Epoch 3/10
6151/6151 [==============================] - 6s 904us/step - loss: 0.2540 - auc: 0.7344
Epoch 4/10
6151/6151 [==============================] - 6s 906us/step - loss: 0.2496 - auc: 0.7489
Epoch 5/10
6151/6151 [==============================] - 6s 899us/step - loss: 0.2471 - auc: 0.7569
Epoch 6/10
6151/6151 [==============================] - 6s 911us/step - loss: 0.2453 - auc: 0.7625
Epoch 7/10
6151/6151 [==============================] - 6s 898us/step - loss: 0.2439 - auc: 0.7670
Epoch 8/10
6151/6151 [==============================] - 6s 898us/step - loss: 0.2427 - auc: 0.7705
Epoch 9/10
6151/6151 [==============================] - 6s 905us/step - loss: 0.2417 - auc: 0.7734
Epoch 10/10
6151/6151 [==============================] - 6s 895us/step - loss: 0.2409 - auc: 0.7758
Epoch 1/

6151/6151 [==============================] - 3s 511us/step - loss: 0.2663 - auc: 0.7411
Epoch 4/10
6151/6151 [==============================] - 3s 513us/step - loss: 0.2559 - auc: 0.7526
Epoch 5/10
6151/6151 [==============================] - 3s 515us/step - loss: 0.2512 - auc: 0.7580
Epoch 6/10
6151/6151 [==============================] - 3s 520us/step - loss: 0.2486 - auc: 0.7614
Epoch 7/10
6151/6151 [==============================] - 3s 517us/step - loss: 0.2471 - auc: 0.7633
Epoch 8/10
6151/6151 [==============================] - 3s 523us/step - loss: 0.2461 - auc: 0.7646
Epoch 9/10
6151/6151 [==============================] - 3s 515us/step - loss: 0.2453 - auc: 0.7658
Epoch 10/10
6151/6151 [==============================] - 3s 522us/step - loss: 0.2448 - auc: 0.7667
Epoch 1/10
6151/6151 [==============================] - 7s 516us/step - loss: 0.4858 - auc: 0.5991
Epoch 2/10
6151/6151 [==============================] - 3s 515us/step - loss: 0.2979 - auc: 0.7065
Epoch 3/10
6151/6151

6151/6151 [==============================] - 4s 699us/step - loss: 0.2559 - auc: 0.7304
Epoch 6/10
6151/6151 [==============================] - 4s 708us/step - loss: 0.2535 - auc: 0.7387
Epoch 7/10
6151/6151 [==============================] - 4s 707us/step - loss: 0.2517 - auc: 0.7445
Epoch 8/10
6151/6151 [==============================] - 4s 708us/step - loss: 0.2503 - auc: 0.7488
Epoch 9/10
6151/6151 [==============================] - 4s 706us/step - loss: 0.2492 - auc: 0.7524
Epoch 10/10
6151/6151 [==============================] - 4s 706us/step - loss: 0.2482 - auc: 0.7554
Epoch 1/10
6151/6151 [==============================] - 8s 696us/step - loss: 0.3561 - auc: 0.5091
Epoch 2/10
6151/6151 [==============================] - 4s 698us/step - loss: 0.2758 - auc: 0.6151
Epoch 3/10
6151/6151 [==============================] - 4s 702us/step - loss: 0.2688 - auc: 0.6672
Epoch 4/10
6151/6151 [==============================] - 4s 700us/step - loss: 0.2639 - auc: 0.6947
Epoch 5/10
6151/6151

6151/6151 [==============================] - 5s 773us/step - loss: 0.2515 - auc: 0.7429
Epoch 8/10
6151/6151 [==============================] - 5s 773us/step - loss: 0.2499 - auc: 0.7479
Epoch 9/10
6151/6151 [==============================] - 5s 781us/step - loss: 0.2487 - auc: 0.7521
Epoch 10/10
6151/6151 [==============================] - 5s 771us/step - loss: 0.2476 - auc: 0.7554
Epoch 1/10
6151/6151 [==============================] - 9s 764us/step - loss: 0.3311 - auc: 0.5485
Epoch 2/10
6151/6151 [==============================] - 5s 755us/step - loss: 0.2892 - auc: 0.6091
Epoch 3/10
6151/6151 [==============================] - 5s 751us/step - loss: 0.2796 - auc: 0.6439
Epoch 4/10
6151/6151 [==============================] - 5s 754us/step - loss: 0.2734 - auc: 0.6674
Epoch 5/10
6151/6151 [==============================] - 5s 751us/step - loss: 0.2690 - auc: 0.6842
Epoch 6/10
6151/6151 [==============================] - 5s 754us/step - loss: 0.2656 - auc: 0.6968
Epoch 7/10
6151/6151

6151/6151 [==============================] - 5s 822us/step - loss: 0.2295 - auc: 0.8078
Epoch 10/10
6151/6151 [==============================] - 5s 820us/step - loss: 0.2278 - auc: 0.8122
Epoch 1/10
6151/6151 [==============================] - 10s 814us/step - loss: 0.2571 - auc: 0.7257
Epoch 2/10
6151/6151 [==============================] - 5s 812us/step - loss: 0.2442 - auc: 0.7675
Epoch 3/10
6151/6151 [==============================] - 5s 816us/step - loss: 0.2406 - auc: 0.7775
Epoch 4/10
6151/6151 [==============================] - 5s 812us/step - loss: 0.2387 - auc: 0.7836
Epoch 5/10
6151/6151 [==============================] - 5s 812us/step - loss: 0.2364 - auc: 0.7893
Epoch 6/10
6151/6151 [==============================] - 5s 820us/step - loss: 0.2346 - auc: 0.7942
Epoch 7/10
6151/6151 [==============================] - 5s 816us/step - loss: 0.2331 - auc: 0.7984
Epoch 8/10
6151/6151 [==============================] - 5s 814us/step - loss: 0.2315 - auc: 0.8029
Epoch 9/10
6151/615

6151/6151 [==============================] - 9s 701us/step - loss: 0.3051 - auc: 0.5948
Epoch 2/10
6151/6151 [==============================] - 4s 708us/step - loss: 0.2634 - auc: 0.6974
Epoch 3/10
6151/6151 [==============================] - 4s 700us/step - loss: 0.2565 - auc: 0.7264
Epoch 4/10
6151/6151 [==============================] - 4s 705us/step - loss: 0.2525 - auc: 0.7412
Epoch 5/10
6151/6151 [==============================] - 4s 708us/step - loss: 0.2498 - auc: 0.7499
Epoch 6/10
6151/6151 [==============================] - 4s 714us/step - loss: 0.2481 - auc: 0.7554
Epoch 7/10
6151/6151 [==============================] - 4s 709us/step - loss: 0.2467 - auc: 0.7597
Epoch 8/10
6151/6151 [==============================] - 4s 707us/step - loss: 0.2457 - auc: 0.7628
Epoch 9/10
6151/6151 [==============================] - 4s 723us/step - loss: 0.2449 - auc: 0.7655
Epoch 10/10
6151/6151 [==============================] - 4s 710us/step - loss: 0.2441 - auc: 0.7675
Epoch 1/10
6151/6151

6151/6151 [==============================] - 5s 802us/step - loss: 0.2701 - auc: 0.6771
Epoch 4/10
6151/6151 [==============================] - 5s 805us/step - loss: 0.2641 - auc: 0.6991
Epoch 5/10
6151/6151 [==============================] - 5s 802us/step - loss: 0.2599 - auc: 0.7137
Epoch 6/10
6151/6151 [==============================] - 5s 812us/step - loss: 0.2568 - auc: 0.7243
Epoch 7/10
6151/6151 [==============================] - 5s 801us/step - loss: 0.2544 - auc: 0.7320
Epoch 8/10
6151/6151 [==============================] - 5s 799us/step - loss: 0.2526 - auc: 0.7384
Epoch 9/10
6151/6151 [==============================] - 5s 811us/step - loss: 0.2511 - auc: 0.7431
Epoch 10/10
6151/6151 [==============================] - 5s 806us/step - loss: 0.2500 - auc: 0.7469
Epoch 1/10
7688/7688 [==============================] - 8s 509us/step - loss: 0.4499 - auc: 0.6176
Epoch 2/10
7688/7688 [==============================] - 4s 508us/step - loss: 0.2818 - auc: 0.7227
Epoch 3/10
7688/7688

In [289]:
print("Best parameters on training set :")
print(nn_clf.best_params_)
print("Best score on training set : {:.3f}".format(nn_clf.best_score_))

Best parameters on training set :
{'learning_rate': 0.0009729020135732503, 'n_hidden': 0, 'n_neurons': 13.122448979591836}
Best score on training set : 0.764


## VI - Compare <a class="anchor" id="17-bullet"></a>

In [290]:
# Compare scores in this iteration

cv_clfs = {
    'LogisticRegression' : LogisticRegression_clf,
    'RidgeClassifier' : RidgeClassifier_clf,
    'KNeighborsClassifier' : KNeighborsClassifier_clf,
    'LinearSVC' : LinearSVC_clf,
    'SVC' : SVC_clf,
    'DecisionTreeClassifier' : DecisionTreeClassifier_clf,
    # 'GradientBoostingClassifier' : GradientBoostingClassifier_clf,
    'RandomForestClassifier' : RandomForestClassifier_clf,
    'NeuralNetwork' : nn_clf,
}

iteration_2 = pd.DataFrame()

for key, clf in cv_clfs.items():
    iteration_2[key] = [clf.best_score_, clf.best_params_]
    
iteration_2.index = ['best_score_ : ' + optimized_metric, 'best_params_']

In [291]:
iteration_2

,LogisticRegression,RidgeClassifier,KNeighborsClassifier,LinearSVC,SVC,DecisionTreeClassifier,RandomForestClassifier,NeuralNetwork
best_score_ : roc_auc,0.768028,0.767798,0.645989,0.768115,0.685251,0.575201,0.728944,0.763832
best_params_,{'model__C': 1.0},{'model__alpha': 10},{'model__n_neighbors': 10},{'model__C': 0.1},{'model__C': 0.4},"{'model__min_samples_leaf': 5, 'model__min_samples_split': 2}",{'model__n_estimators': 500},"{'learning_rate': 0.0009729020135732503, 'n_hidden': 0, 'n_neurons': 13.122448979591836}"


In [292]:
iteration_2.iloc[0].sort_values(ascending=False)

LinearSVC                 0.768115
LogisticRegression        0.768028
RidgeClassifier           0.767798
NeuralNetwork             0.763832
RandomForestClassifier    0.728944
SVC                       0.685251
KNeighborsClassifier      0.645989
DecisionTreeClassifier    0.575201
Name: best_score_ : roc_auc, dtype: object

In [293]:
iteration_2.to_csv('./Scores/iteration_2.csv')

# 6. Selected model fine tuning and interpretation <a class="anchor" id="18-bullet"></a>

## I - Fine tuning <a class="anchor" id="19-bullet"></a>

### a. With lbfgs solver

In [297]:
# Test 1 : same test as model selection with more parameters

model = LogisticRegression(random_state=random_state, max_iter=10000, 
                           multi_class='ovr', penalty='l2', solver='lbfgs')
param_grid = {
    'C' : np.linspace(0.1, 10, num=10),
    'class_weight' : [None, 'balanced']
}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf_1 = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters on training set :
{'model__C': 2.3000000000000003, 'model__class_weight': None}
Best score on training set : 0.769
Proceed LogisticRegression - done in 248s


In [299]:
# Test 2 : we keep class_weight=None and test other values of C centered in our result

model = LogisticRegression(random_state=random_state, max_iter=10000, 
                           multi_class='ovr', penalty='l2', solver='lbfgs', class_weight=None)
param_grid = {
    'C' : np.linspace(1.5, 3.5, num=10),
}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf_2 = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters on training set :
{'model__C': 1.9444444444444444}
Best score on training set : 0.768
Proceed LogisticRegression - done in 122s


### b. With saga solver

In [301]:
# Test 3 : solver saga with different penalties ratios 

model = LogisticRegression(random_state=random_state, max_iter=10000, 
                           multi_class='ovr', penalty='elasticnet', solver='saga', class_weight=None)
param_grid = {
    'C' : np.linspace(1, 5, num=5),
    'l1_ratio' : np.linspace(0, 1, num=11)
}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf_3 = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 55 candidates, totalling 275 fits
Best parameters on training set :
{'model__C': 4.0, 'model__l1_ratio': 0.1}
Best score on training set : 0.768
Proceed LogisticRegression - done in 20467s


In [304]:
# Test 4 : centered search on previous results

model = LogisticRegression(random_state=random_state, max_iter=10000, 
                           multi_class='ovr', penalty='elasticnet', solver='saga', class_weight=None)
param_grid = {
    'C' : np.linspace(3.5, 4.5, num=3),
    'l1_ratio' : np.linspace(0.05, 0.15, num=3)
}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf_4 = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters on training set :
{'model__C': 4.5, 'model__l1_ratio': 0.1}
Best score on training set : 0.768
Proceed LogisticRegression - done in 4418s


### c. With liblinear solver

In [305]:
# Test 5 : solver liblinear with l1 and l2 penalties 

model = LogisticRegression(random_state=random_state, max_iter=10000, 
                           multi_class='ovr', solver='liblinear', class_weight=None)
param_grid = {
    'C' : np.linspace(0.1, 10, num=10),
    'penalty' : ['l1', 'l2']
}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf_5 = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters on training set :
{'model__C': 3.4000000000000004, 'model__penalty': 'l2'}
Best score on training set : 0.768
Proceed LogisticRegression - done in 716s


In [307]:
# Test 6 : centered search on previous results 

model = LogisticRegression(random_state=random_state, max_iter=10000, 
                           multi_class='ovr', solver='liblinear', class_weight=None, penalty='l2')
param_grid = {
    'C' : np.linspace(2.5, 4.5, num=5),
}

with timer("Proceed LogisticRegression"):
    LogisticRegression_clf_6 = run_GridSearchCV(model, x_train_pca, y_train, folds, 
                                              param_grid, optimized_metric, balance_class=balance_class)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best parameters on training set :
{'model__C': 3.5}
Best score on training set : 0.768
Proceed LogisticRegression - done in 141s


### d. Compare

In [308]:
# Compare scores in this iteration

models = [LogisticRegression_clf_1, LogisticRegression_clf_2, LogisticRegression_clf_3,
LogisticRegression_clf_4, LogisticRegression_clf_5, LogisticRegression_clf_6]

fine_tuning = pd.DataFrame()

for i, model in enumerate(models):
    fine_tuning['LogisticRegression_clf_{}'.format(i+1)] = [model.best_score_, model.best_params_]
    
fine_tuning.index = ['best_score_ : ' + optimized_metric, 'best_params_']

In [313]:
fine_tuning.iloc[0].sort_values(ascending=False)

LogisticRegression_clf_1    0.768558
LogisticRegression_clf_3    0.768387
LogisticRegression_clf_5     0.76834
LogisticRegression_clf_2    0.768197
LogisticRegression_clf_6    0.768014
LogisticRegression_clf_4    0.767996
Name: best_score_ : roc_auc, dtype: object

Valeurs testées première itération (cleaned_data_1):
- LogisticRegression : {'C' : np.linspace(0.1, 1, num=4)}, best : C = 0.1 -> tester plus petit
- RidgeClassifier : {'alpha' : np.linspace(1, 10, num=4, dtype=int)}, best : alpha = 10 -> tester plus grand
- KNeighborsClassifier : {'n_neighbors' : np.linspace(3, 10, num=4, dtype=int)}, best : n_neighbors = 10 > tester plus grand
- LinearSVC : {'penalty' : ['l1', 'l2'], 'C' : np.linspace(0.1, 1, num=4)}, bests :
    - C = 0.7 --> tester valeurs autour
    - penalty = l2 --> conserver
- SVC : {'C' : np.linspace(0.1, 1, num=4)}, best : C = 0.4 -> tester valeurs autour
- DecisionTreeClassifier : {'min_samples_split' : [2, 4, 8], 'min_samples_leaf' : [1, 3, 5]}, bests :
    - min_samples_split = 2 --> conserver
    - min_samples_leaf = 5 --> tester plus grand
- GradientBoostingClassifier : {'n_estimators' : [10, 100, 500]}, best : n_estimators = 500 -> tester plus grand
- RandomForestClassifier : {'n_estimators' : [10, 100, 500]}, best : n_estimators = 500 -> tester plus grand